<a href="https://colab.research.google.com/github/Violeta759/Clase5/blob/main/Gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Yahoo finance](https://finance.yahoo.com/)

Librería:[yfinance](https://algotrading101.com/learn/yfinance-guide/)

In [ ]:
!pip install -U yfinance pandas_datareader

# Datos: Serie multivariante.

Valor de cierre del IBEX, BBVA y TELEFÓNICA(TEF).

In [ ]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
yf.pdr_override() # <== that's all it takes :-)
data = yf.download("BBVA TEF ^IBEX", start="2020-01-01", end="2020-10-30")
data.head()

Para quitar los **NAN** sin tener que eliminarlos (porque a veces son importantes), les danis un valor. En este caso el valor se lo damos a partir de una interpolación lineal. 

(Hay muchas formas de darles valores).

In [ ]:
d=data.loc[:,'Close'].interpolate(limit=2, limit_direction='backward') 
d.head()

## IBEX depende de la semana anterior de BBVA y TELFÓNICA:

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#Escalanos los datos
dScale = MinMaxScaler()
dScale.fit(d.values)
ldatos=dScale.transform(d.values)

In [ ]:
pd.DataFrame(ldatos,columns=['BBVA','TEF','^IBEX']).plot(figsize=(15,5))

In [ ]:
X,Y=ldatos[:,0:2],ldatos[:,2]
#ldatos[:,0:2] -> De cada fila quiero las columnas 0-1-2.

In [ ]:
def windowData(Xi,Yi,window_input=1,window_output=1,step=1):
  Xaux=[]
  Yaux=[]
  #Datos necesarios:
  dn=window_input+window_output
  #Calcular cuantos pasos completos podemos realizar, sin contar el último bloque [input,output].
  ld=int((len(Xi)-dn)/step)*step
  for i in range(len(Xi)-ld-1,len(Xi)-dn,step):
    Xaux.append(Xi[i:i+window_input])
    Yaux.append(Yi[i+window_input:i+dn])
  return Xaux,Yaux

In [ ]:
#Contruimos los conjunto de datos X e Y:
Xw,Yw=windowData(X,Y,window_input=20,window_output=1,step=1)
Xw,Yw=np.array(Xw),np.array(Yw)
Xw.shape
#Cada elemento de esos 20 van a ser dos valores (los de BBVA y Telefónica).

Ya tenemos los datos de entrada construidos.

## Librerías necesarias:
- [Keras](https://keras.io/).
- [Tensorflow](https://www.tensorflow.org/).

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU
from tensorflow.keras import metrics

## Definimos la Red:

Definimos nuestro perceptron con las siguientes capas (**layers**):
- Modelo secuencial [Sequential](https://keras.io/api/models/sequential/).
- Capa de entrada: [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
- Capa Densa: [Dense](https://keras.io/api/layers/core_layers/dense/) .

La capa **SimpleRNN** define una unidad recurente con **unit** neuronas de salida y una entrada de **input_shape**.

**Recurrente básica**:

In [ ]:
gru = Sequential()
gru.add(GRU(units=34, activation='relu',input_shape=(20,2))) #añadimos una capa de recurrencia. Función de activación -> activation.
gru.add(Dense(1))
gru.compile(loss='mean_squared_error', optimizer='rmsprop')
gru.summary()

Cuantos menos datos entrenemos más rápido. 
(Non-trainable params: 0).

In [ ]:
h=gru.fit(Xw,Yw,validation_split=0.30, epochs=15,batch_size=14)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Esto sale porque tenemos pocas épocas (antes apróx. 100 épocas, y ahora 15), pero aún así obtenemos buenos resultados. 

## Utilizamos la red:
Utilizar el perceptron aprendido con el método [**predict**](https://keras.io/api/models/model_training_apis/#predict-method).

In [ ]:
Y0=gru.predict(Xw)

In [ ]:
pd.DataFrame(np.hstack([Yw,Y0]),columns=['P','R']).plot(figsize=(15,5))

(Predice el valor del IBEX según los valores de Telefónica y BBVA de los 20 días anteriores).

Aquí se ven muy bien las tendencias, son resultados más suaves que los anteriores. Depende de nuestros objetivos un modelo es bueno o es malo. Por lo que en comparación con los otros modelos, aunque este generalice más, no es malo.

## Salvar y Cargar Redes:
- Salvar mediante el método [**save**](https://keras.io/api/models/model_saving_apis/#save-method).
- Recuperar un modelo con el método [**load_model**](keras.models.load_model).

(Guardo mi red).

In [ ]:
gru.save('gru.h5')
gru2=keras.models.load_model('gru.h5')